#### Introduction
The analysis will use two dataframes: 'site_indices' which contains all the species 
abundance records for each year, and 'site_locations' which contains all locational
data relating to each survey. Both datasets have already been cleaned. In this section 
they will be joined into one master dataframe: 'ukbms_master_v1'.

#### Workflow
1) The dataframes are merged
2) Null values are removed
3) Data types are changed
4) String characters are modified for consistency
5) The formatting is adjsuted for imporved workability. 

In [2]:
# Importing the required packages
import pandas as pd
import os
from pathlib import Path

# Importing localised file directory
project_root = Path(os.environ['butterfly_project'])

# Importing the datasets
site_indices = pd.read_csv(
    project_root/'Data'/'UKBMS'/'ukbms_site_indices'/'ukbms_site_indices_cleaned_v1.csv',
    index_col=0
)

site_locations = pd.read_csv(
    project_root/'Data'/'UKBMS'/'ukbms_site_location_data'/'ukbms_site_location_cleaned_v1',
    index_col=0
)

#### Merging Dataframes

In [3]:
# The dataframes are joined using the site location number. In 'site_indices' this is 
# called 'SITE_INDEX'. In 'site_locations', 'SITE_NAME' is used. 
ukbms_merge = (
    site_indices
    .merge(site_locations, 
           left_on='SITE_CODE', 
           right_on='Site_Number', 
           how='left')
    .drop(columns=['Site_Number','Country']) # removing redundant columns
)

#### Removing Null Values

In [4]:
# Checking for null values
print(ukbms_merge.isna().sum())

# 16406 records from the 'site_location' dataframe are null. 
# This suggests some 'SITE_CODES' in the 'site_indices' dataframe do not match.
# The ukbms keeps a number of 'sensitive' site locations confidential. This is 
# the most likely cause of the missing data. 

SITE_CODE            0
COUNTRY              0
SPECIES_CODE         0
SPECIES              0
COMMON_NAME          0
YEAR                 0
SITE_INDEX           0
Site_Name        16406
Gridreference    16406
Easting          16406
Northing         16406
Survey_type      16406
dtype: int64


In [5]:
# Computing the number of missing rows (prefferred to manual input in case dataset is 
# updated).
rows_missing = len(
    ukbms_merge[ukbms_merge['Site_Name'].isna()]
)

# Computing the percentage of null values. 
percent_rows_missing = round(
    (rows_missing/len(ukbms_merge))*100
    ,2)

print(
    'Rows with no location data: ' + str(percent_rows_missing) + '%'
)

# The percentage of missing rows makes up a small proportion of the total data. This 
# will likely have little to no effect on the analysis. 

Rows with no location data: 3.04%


In [6]:
# Removing nulls
ukbms_nulls_removed = ukbms_merge.copy()

# Removing null rows from dataset
ukbms_nulls_removed = (
    ukbms_nulls_removed
    .dropna() # removes all rows with null values
    .reset_index(drop=True) # new index required follwoing row removal
)

#### Correcting Data Types

In [7]:
# Cleaning data types
ukbms_data_type = ukbms_nulls_removed.copy()

# Converting Easting/Northing data types from string to integer
ukbms_data_type[['Easting', 'Northing']] = (
    ukbms_nulls_removed[['Easting', 'Northing']]
    .astype(int)
)

#### Making String Characters Consistent

In [8]:
# Making string character case consistent
ukbms_lowercase = ukbms_data_type.copy()

# Converting all uppercase characters in dataframe records to lowercase
ukbms_lowercase = (
    ukbms_lowercase.apply(
        lambda x: x.str.lower() 
        if x.dtype=='object' else x # function is not applied to integer data types
    )
)

# Converting all characters in column headings to lowercase
ukbms_lowercase.columns = ukbms_lowercase.columns.str.lower()

#### Adjusting Formatting

In [9]:
# Rearranging columns and sorting rows
ukbms_format = ukbms_lowercase.copy()
ukbms_format = (
    ukbms_format[['site_code', 
                  'year', 
                  'country', 
                  'site_name', 
                  'species_code', 
                  'species', 
                  'common_name', 
                  'site_index', 
                  'gridreference', 
                  'easting', 
                  'northing']]
    .sort_values(['site_code', 'year'])
    .reset_index(drop=True)
)

In [10]:
# exporting to csv
ukbms_format.to_csv(project_root/'Data'/'UKBMS'/'ukbms_master_v1.csv')